In [59]:
import requests
import re
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [60]:
# nltk download english stopwords
nltk.download('stopwords')

nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [72]:
PROBLEM = "Problem"
USER_CHARECTER = "Interviewee Characteristics"
QUESTION_AND_OBJ = "Questions and Follow Up Criterias"
# QUESTION_PREFIX = "Question: "
# OBJ_PREFIX = "Motivation: "
CONVO = "Conversation"
OBJ = "Follow Up Criterias"
QUESTION = "Question"

In [97]:
test_data = None
with open("bench_persona_qs_goals_with_conversations_yash.json", "r") as f:
    test_data = json.load(f)
test_data[0]

{'Problem': 'Lack of access to clean drinking water in rural areas',
 'Interviewee Characteristics': {'Name': 'Rajesh',
  'Age': 45,
  'Gender': 'Male',
  'Income': 'Low',
  'Education': 'Primary',
  'Location': 'Rajasthan'},
 'Questions and Follow Up Criterias': [{'Question': 'Do you have access to clean drinking water?',
   'Follow Up Criterias': ['To know if the person is affected by the problem or not.']},
  {'Question': 'How far do you have to travel to get clean drinking water?',
   'Follow Up Criterias': ['To know the distance the person has to travel.',
    'Was the user answer specific?']},
  {'Question': 'Do you have to pay for clean drinking water?',
   'Follow Up Criterias': ['To know if the person has to pay for water.',
    'Did the user answer the question directly?']}],
 'Conversation': [{'Interviewer Response': 'Hello Rajesh, thank you for taking the time to speak with me today. Can you tell me if you have access to clean drinking water?',
   'Interviewee Response': 'Y

In [94]:
stop_words = set(stopwords.words('english') + ["Do", "Can", "What", "How", "Why", "Where", "When", "Which", "Who", "Whom", "Whose"])
stemmer = nltk.stem.LancasterStemmer()
def stopwordremoved(s):
    # remove punctuations
    s = re.sub(r'[^\w\s]','',s)
    s = stemmer.stem(s)
    print(s)
    word_tokens = word_tokenize(s)
    # make stem words
    # converts the words in word_tokens to lower case and then checks whether 
    #they are present in stop_words or not
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    #with no lower case conversion
    filtered_sentence = []
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

In [95]:
stopwordremoved("Do you have access to clean drinking water?")

do you have access to clean drinking water


['access', 'clean', 'drinking', 'water']

In [98]:
# {'Problem': 'Lack of access to clean drinking water in rural areas',
#  'Interviewee Characteristics': {'Name': 'Rajesh',
#   'Age': 45,
#   'Gender': 'Male',
#   'Income': 'Low',
#   'Education': 'Primary',
#   'Location': 'Rajasthan'},
#  'Questions and Follow Up Criterias': [{'Question': 'Do you have access to clean drinking water?',
#    'Follow Up Criterias': ['To know if the person is affected by the problem or not.']},
#   {'Question': 'How far do you have to travel to get clean drinking water?',
#    'Follow Up Criterias': ['To know the distance the person has to travel.',
#     'Was the user answer specific?']},
#   {'Question': 'Do you have to pay for clean drinking water?',
#    'Follow Up Criterias': ['To know if the person has to pay for water.',
#     'Did the user answer the question directly?']}],
#  'Conversation': [{'Interviewer Response': 'Hello Rajesh, thank you for taking the time to speak with me today. Can you tell me if you have access to clean drinking water?',
#    'Interviewee Response': 'Yes, we have a well nearby.',
#    'Fixed Question': True},
#   {'Interviewer Response': "That's good to hear. Can you tell me how far you have to travel to get clean drinking water?",
#    'Interviewee Response': 'We have to walk for about 2 kilometers to reach the well.',
#    'Fixed Question': True},
#   {'Interviewer Response': 'I see. And do you have to pay for clean drinking water?',
#    'Interviewee Response': "No, we don't have to pay for the water from the well.",
#    'Fixed Question': True},
#   {'Interviewer Response': "That's great to hear. Can you tell me if there are any challenges you face in accessing clean drinking water?",
#    'Interviewee Response': 'Sometimes during summers, the well dries up and we have to go to a nearby village to get water.',
#    'Fixed Question': True},
#   {'Interviewer Response': 'I understand. Can you tell me more about the challenges you face during summers?',
#    'Interviewee Response': 'The water from the nearby village is not always clean and we have to boil it before drinking.',
#    'Fixed Question': False},
#   {'Interviewer Response': 'I see. And have you faced any health issues due to drinking unclean water?',
#    'Interviewee Response': 'Yes, my daughter fell sick last year due to drinking contaminated water.',
#    'Fixed Question': True},
#   {'Interviewer Response': "I'm sorry to hear that. Can you tell me more about the health issues your daughter faced?",
#    'Interviewee Response': 'She had diarrhea and was very weak for a few days.',
#    'Fixed Question': False},
#   {'Interviewer Response': 'I understand. Thank you for sharing your experiences with me today, Rajesh. It was very helpful.',
#    'Interviewee Response': '',
#    'Fixed Question': False}]}

for survey in test_data:
    convo = survey[CONVO]
    questions = survey[QUESTION_AND_OBJ]
    # Using tf idf figure out which questions are is asked in each element of the conversation, if none are asked then set 'Fixed Question': False else set 'Question Index' as index of question in the list
    for i, c in enumerate(convo):
        if not c['Fixed Question']:
            continue
        c['Fixed Question'] = False
        for j, q in enumerate(questions):
            # Use condition on how many words (after removing stopwords) are in common divided by the total number of words (after removing stopwords) in the question
            question_words = stopwordremoved(q['Question'])
            interviewer_response_words = stopwordremoved(c['Interviewer Response'])
            if len(set(question_words) & set(interviewer_response_words)) / len(question_words) > 0.8:
                c['Fixed Question'] = True
                c['Question Index'] = j
                break

test_data[0]

do you have access to clean drinking water
hello rajesh thank you for taking the time to speak with me today can you tell me if you have access to clean drinking water
do you have access to clean drinking water
thats good to hear can you tell me how far you have to travel to get clean drinking water
how far do you have to travel to get clean drinking water
thats good to hear can you tell me how far you have to travel to get clean drinking water
do you have access to clean drinking water
i see and do you have to pay for clean drinking water
how far do you have to travel to get clean drinking water
i see and do you have to pay for clean drinking water
do you have to pay for clean drinking water
i see and do you have to pay for clean drinking water
do you have access to clean drinking water
thats great to hear can you tell me if there are any challenges you face in accessing clean drinking water
how far do you have to travel to get clean drinking water
thats great to hear can you tell me 

{'Problem': 'Lack of access to clean drinking water in rural areas',
 'Interviewee Characteristics': {'Name': 'Rajesh',
  'Age': 45,
  'Gender': 'Male',
  'Income': 'Low',
  'Education': 'Primary',
  'Location': 'Rajasthan'},
 'Questions and Follow Up Criterias': [{'Question': 'Do you have access to clean drinking water?',
   'Follow Up Criterias': ['To know if the person is affected by the problem or not.']},
  {'Question': 'How far do you have to travel to get clean drinking water?',
   'Follow Up Criterias': ['To know the distance the person has to travel.',
    'Was the user answer specific?']},
  {'Question': 'Do you have to pay for clean drinking water?',
   'Follow Up Criterias': ['To know if the person has to pay for water.',
    'Did the user answer the question directly?']}],
 'Conversation': [{'Interviewer Response': 'Hello Rajesh, thank you for taking the time to speak with me today. Can you tell me if you have access to clean drinking water?',
   'Interviewee Response': 'Y

In [104]:
# with open("bench_persona_qs_goals_with_conversations_yash_parsed.json", "w") as f:
#     json.dump(test_data, f, indent=4)

In [68]:
def generate_response(prompt, max_new_tokens=200):
    # Set the API endpoint URL
    api_url = "http://localhost:8000/generate_follow_up/"  # Update with your actual server URL

    # Prepare the request payload
    prompt_data = {"chat_content": prompt, "max_new_tokens" : max_new_tokens}

    # Send the POST request
    response = requests.post(api_url, json=prompt_data)

    if response.status_code == 200:
        # The request was successful
        generated_text = response.json()["generated_follow_up"]
        # print(f"Generated Text: {generated_text}")
        return generated_text
    else:
        # Something went wrong
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [112]:
prompt_template = \
    """
    You are an expert interviewer, your job is to generate 1 followup question based interview till now and the objective of the main question not been met till now.
    Do not generate answers. Be empathic to user. 
    
    The following are details you will use
    
    Main question: {main_question}
    Objectives left: {objectives_left}
    
    Chat history:-
    {chat_history}
    
    Generate exactly one follow-up question comes from the assistant to be readily sent to user in a friendly tone.
    Do not generate the questions that's already present in the chat history, however you can use different words in case of any ties
    """

def question_generation_agent(chat_history, main_question, objectives_left, user_charecteristics="calm and patient"):

    # fetch llm response
    prompt = prompt_template.format(chat_history=chat_history, main_question=main_question, objectives_left=objectives_left)
    response =  generate_response(prompt)

    # Use regular expressions to parse the llm output
    end_of_turn_tags = re.findall(r'<start_of_turn>model(.*?)<eos>', response, re.DOTALL)
    parsed_response = end_of_turn_tags[0].strip().strip('"').strip("'")

    # print("Parsed LLM response :", parsed_response)
    return parsed_response

In [70]:
test_data[1]

{'Problem': 'Lack of access to healthcare in rural areas',
 'Interviewee Characteristics': {'Name': 'Amit',
  'Age': 35,
  'Gender': 'Male',
  'Income': 'Low',
  'Education': 'Primary',
  'Location': 'Uttar Pradesh'},
 'Questions and Follow Up Criterias': [{'Question': 'Do you have access to healthcare?',
   'Follow Up Criterias': ['To know if the person has access to healthcare or not.']},
  {'Question': 'How far do you have to travel to get to a hospital?',
   'Follow Up Criterias': ['To know the distance the person has to travel.',
    'Was the user answer specific?']},
  {'Question': 'Do you have to pay for healthcare?',
   'Follow Up Criterias': ['To know if the person has to pay for healthcare.',
    'Did the user answer the question directly?']},
  {'Question': 'Do you think the quality of healthcare is good?',
   'Follow Up Criterias': ["To know the person's opinion on the quality of healthcare.",
    'Did the user answer the question directly?']}],
 'Conversation': [{'Intervie

In [102]:
def transform_conversation(convo):
    chat_history = ""
    for item in convo:
        if item['Interviewer Response'] and len(item['Interviewee Response']) > 0:
            chat_history += "Interviewer: " + item['Interviewer Response'] + "\n"
        if item['Interviewee Response'] and len(item['Interviewee Response']) > 0:
            chat_history += "Interviewee: " + item['Interviewee Response'] + "\n"
    return chat_history

In [107]:
# Separate input to variables
i = 1
ci = 1
qi = 2
conversation = transform_conversation(test_data[i][CONVO][2:3])
main_question = test_data[ci][QUESTION_AND_OBJ][qi][QUESTION]
objectives_left = test_data[ci][QUESTION_AND_OBJ][qi][OBJ]
user_characteristics = test_data[i][USER_CHARECTER]

# Print variables
print("conversation:", conversation)
print("main_question:", main_question)
print("objectives_left:", objectives_left)
print("user_characteristics:", user_characteristics)

print("----")

# Call the function
question_generation_agent(conversation, main_question, objectives_left, user_characteristics)


conversation: Interviewer: Can you tell me how far exactly?
Interviewee: I have to travel around 20 kilometers to get to a hospital.

main_question: Do you have to pay for healthcare?
objectives_left: ['To know if the person has to pay for healthcare.', 'Did the user answer the question directly?']
user_characteristics: {'Name': 'Amit', 'Age': 35, 'Gender': 'Male', 'Income': 'Low', 'Education': 'Primary', 'Location': 'Uttar Pradesh'}
----


Parsed LLM response : Sure, here is the follow-up question:

"Given that you have to travel 20 kilometers to get to a hospital, do you typically incur any costs for transportation when seeking healthcare?


'Sure, here is the follow-up question:\n\n"Given that you have to travel 20 kilometers to get to a hospital, do you typically incur any costs for transportation when seeking healthcare?'

In [110]:
output_data = []
for i, survey in enumerate(test_data):
    convo = survey[CONVO]
    for j, c in enumerate(convo):
        if not c['Fixed Question']:
            continue
        qi = c['Question Index']
        for k in range(j, len(convo)):
            if j!=k and convo[k]['Fixed Question']:
                break
            conversation = transform_conversation(survey[CONVO][j:k+1])
            main_question = survey[QUESTION_AND_OBJ][qi][QUESTION]
            objectives_left = survey[QUESTION_AND_OBJ][qi][OBJ]
            user_characteristics = survey[USER_CHARECTER]

            follow_up_question = question_generation_agent(conversation, main_question, objectives_left, user_characteristics)
            output_data.append({"conversation": conversation, "main_question": main_question, "objectives_left": objectives_left, "user_characteristics": user_characteristics, "follow_up_question": follow_up_question})
    print("done: ", i)


with open("qg_promt1.json", "w") as f:
    json.dump(output_data, f, indent=4)

Parsed LLM response : Sure, here is the follow-up question:

"Would you be able to describe the quality of the water from your well, and if it's treated in any way?"

This question is designed to further understand the accessibility and quality of the clean drinking water available to the interviewee.
Parsed LLM response : Sure, here is the follow-up question:

"Could you provide more details about the 2 kilometers you mentioned? Is the distance straight-line or does it involve any winding paths?
Parsed LLM response : Sure, here is the follow-up question:

"Is the well water treated to make it clean and safe to drink?"

This question is designed to further clarify the user's answer and ensure that the objective of the main question ("To know if the person has to pay for water") has not been met.
Parsed LLM response : Sure, here is the follow-up question:

"Given that the well sometimes dries up during summers, have you ever encountered any difficulties in accessing clean drinking water